## The engine of neural networks: gradient-based optimization

一个优化训练循环（training  loop）内，其具体过程如下。必要时一直重复这些步骤。

(1)抽取训练样本x和对应目标y组成的数据批量。

(2)在x上运行网络［这一步叫作前向传播（forward    pass）］，得到预测值y_pred。

(3)计算网络在这批数据上的损失，用于衡量y_pred和y之间的距离。

(4)更新网络的所有权重，使网络在这批数据上的损失略微下降。

方法：
1. 保持网络中其他权重不变，只考虑某个标量系数，让其尝试不同的取值。
2. 一种更好的方法是利用网络中所有运算都是可微（differentiable）的这一事实，计算损失相对于网络系数的梯度（gradient），然后向梯度的反方向改变系数，从而使损失降低

### 2.4.1 What's a derivative?
什么是导数

连续的光滑函数f(x) = y，将实数x映射为另一个实数y,在某点p附近，当x足够接近p时，斜率a被称为f在p点的导数（derivative）。  
如果a是负的，说明x在p点附近的微小变化将导致f(x)减小；  
如果a是正的，那么x的微小变化将导致f(x)增大。 
此外，a的绝对值（导数大小）表示增大或减小的速度快慢 

### 2.4.2 Derivative of a tensor operation: the gradient
张量运算的导数：梯度

梯度是张量运算的导数，是将导数这一概念向多元函数导数推广的过程。

### 2.4.3 Stochastic gradient descent
随机梯度下降

函数的最小值是导数为 0的点，因此你只需找到所有导数为 0的点，然后计算函数在其中哪个点具有最小值。  
(1)抽取训练样本x和对应目标y组成的数据批量。

(2)在x上运行网络，得到预测值y_pred。

(3)计算网络在这批数据上的损失，用于衡量y_pred和y之间的距离。

(4)计算损失相对于网络参数的梯度［一次反向传播（backward  pass）］。

(5)将参数沿着梯度的反方向移动一点，比如W  -= step  * gradient，从而使这批数据上的损失减小一点。

step (learning rate)如果取值太小，则沿着曲线的下降需要很多次迭代，而且可能会陷入局部极小点。如果取值太大，则更新权重值之后可能会出现在曲线上完全随机的位置。

stochastic gradient descent，SGD随机梯度下降。  
SGD还有多种变体，其区别在于计算下一次权重更新时还要考虑上一次权重更新，而不是仅仅考虑当前梯度值，  
比如带动量的SGD、Adagrad、RMSProp等变体。这些变体被称为优化方法（optimization method）或优化器（optimizer）。  
其中动量的概念尤其值得关注，它在许多变体中都有应用。  
动量解决了SGD的两个问题：收敛速度和局部极小点。  

这一方法的灵感来源于物理学。有一种有用的思维图像，就是将优化过程想象成一个小球从损失函数曲线上滚下来。如果小球的动量足够大，那么它不会卡在峡谷里，最终会到达全局最小点。动量方法的实现过程是每一步都移动小球，不仅要考虑当前的斜率值（当前的加速度），还要考虑当前的速度（来自于之前的加速度）。这在实践中的是指，更新参数w不仅要考虑当前的梯度值，还要考虑上一次的参数更新，其简单实现如下所示。



In [ ]:
w #参数
past_velocity=0
momentum=0.1 #不变的动量因子
while loss > 0.01: #优化循环
    w,loos,gradient=get_current_parameters()
    velocity=past_velocity*momentum-learning_rate*gradient
    w = w+momentum*velocity-learning_rate*gradient
    past_velocity=velocity
    update_parameter(w)

### 2.4.4 Chaining derivatives: The Backpropagation algorithm
链式求导:反向传播算法

f包含3个张量运算a,b,c;还有3个权重矩阵W1,W2和W3  
f(W1, W2, W3) = a(W1, b(W2, c(W3)))  
这种函数链可以利用下面这个恒等式进行求导，它称为链式法则（chain rule）：(f(g(x)))’ =  f'(g(x)) * g'(x)。  
将链式法则应用于神经网络梯度值的计算，得到的算法叫作反向传播（backpropagation，有时也叫反式微分，reverse-mode differentiation）。  
反向传播从最终损失值开始，从最顶层反向作用至最底层，利用链式法则计算每个参数对损失值的贡献大小。

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [7]:
x = tf.Variable(0.)
with tf.GradientTape() as tape: # 求导函数
    y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)

In [25]:
x = tf.Variable(tf.random.uniform((2, 2)))
with tf.GradientTape() as tape:
    y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)

In [26]:
W = tf.Variable(tf.random.uniform((2, 2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2, 2))
with tf.GradientTape() as tape:
    y = tf.matmul(x, W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])

### 2.5 Looking back at our first example
回顾第一个例子

In [29]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
#输入图像保存在  float32格式的   Numpy张量中，形状分别为  (60000,784)（训练数据）和(10000, 784)（测试数据）。

In [32]:
#构建网络
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])
#现在你明白了，这个网络包含两个Dense层，每层都对输入数据进行一些简单的张量运算，这些运算都包含权重张量。
#权重张量是该层的属性，里面保存了网络所学到的知识（knowledge）。

In [33]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
#现在你明白了，categorical_crossentropy是损失函数，是用于学习权重张量的反馈信号，在训练阶段应使它最小化。
#你还知道，减小损失是通过小批量随机梯度下降来实现的。梯度下降的具体方法由第一个参数给定，即rmsprop优化器。

In [34]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)
#网络开始在训练数据上进行迭代（每个小批量包含128个样本），共迭代  5次［在所有训练数据上迭代一次叫作一个轮次（epoch）］。
#在每次迭代过程中，网络会计算批量损失相对于权重的梯度，并相应地更新权重。   
#5轮之后，网络进行了2345次梯度更新（每轮469次），网络损失值将变得足够小，使得网络能够以很高的精度对手写数字进行分类。

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.4313 - accuracy: 0.8744
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1132 - accuracy: 0.9669
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0720 - accuracy: 0.9780
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0500 - accuracy: 0.9852
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0352 - accuracy: 0.9896


### 本章小结

1. 学习是指找到一组模型参数，使得在给定的训练数据样本和对应目标值上的损失函数最小化。  
2. 学习的过程：随机选取包含数据样本及其目标值的批量，并计算批量损失相对于网络参数的梯度。随后将网络参数沿着梯度的反方向稍稍移动（移动距离由学习率指定）。
3. 整个学习过程之所以能够实现，是因为神经网络是一系列可微分的张量运算，因此可以利用求导的链式法则来得到梯度函数，这个函数将当前参数和当前数据批量映射为一个梯度值。
4. 后续几章你会经常遇到两个关键的概念：损失和优化器。将数据输入网络之前，你需要先定义这二者。
5. 损失是在训练过程中需要最小化的量，因此，它应该能够衡量当前任务是否已成功解决。
6. 优化器是使用损失梯度更新参数的具体方式，比如     RMSProp优化器、带动量的随机梯度下降（SGD）等。